# Ensemble Model
Putting together an ensemble of the average wordvec model and a model using the other features.

# Reading the data

In [1]:
import pandas as pd
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')

In [2]:
wordvec_train = pd.read_pickle('train_wordvec.pickle')
wordvec_test = pd.read_pickle('test_wordvec.pickle')

In [3]:
train = train.merge(wordvec_train, on=['id'])
test = test.merge(wordvec_test, on=['id'])
train.head()

,id,keyword,location,text,target,cleaned_text,wordvec,keyword_wordvec
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[-0.26623327, 0.05843069, -0.1404636, -0.05265...","[-0.26623327, 0.05843069, -0.1404636, -0.05265..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,"[-0.025449565, 0.031005142, -0.15566371, -0.23...","[-0.025449565, 0.031005142, -0.15566371, -0.23..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,"[0.0059339865, 0.016337818, -0.105279535, -0.0...","[0.0059339865, 0.016337818, -0.105279535, -0.0..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...,"[-0.18147185, 0.20731743, 0.014147284, -0.2182...","[-0.18147185, 0.20731743, 0.014147284, -0.2182..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[-0.06394094, -0.01423019, 0.0063574947, 0.071...","[-0.06394094, -0.01423019, 0.0063574947, 0.071..."


In [4]:
train_tree = pd.read_csv('train_metafeatures_tree.csv')
test_tree = pd.read_csv('test_metafeatures_tree.csv')
train_normalized = pd.read_csv('train_metafeatures_normalized.csv')
test_normalized = pd.read_csv('test_metafeatures_normalized.csv')
train_normalized.head()

,id,num_char_n,num_words_n,num_hash_n,num_mention_n,num_url_n,has_location,geocoded,longitude_n,latitude_n,target
0,1,0.433121,0.419355,0.076923,0.0,0.0,False,False,0.0,0.0,1
1,4,0.242038,0.225806,0.000000,0.0,0.0,False,False,0.0,0.0,1
2,5,0.847134,0.709677,0.000000,0.0,0.0,False,False,0.0,0.0,1
3,6,0.356688,0.225806,0.076923,0.0,0.0,False,False,0.0,0.0,1
4,7,0.541401,0.516129,0.153846,0.0,0.0,False,False,0.0,0.0,1


In [5]:
train_tree = train_tree.merge(train, on=['id','target'])
train_normalized = train_normalized.merge(train, on=['id','target'])
test_tree = test_tree.merge(test, on=['id'])
test_normalized = test_normalized.merge(test, on = ['id'])
train_normalized.head()

,id,num_char_n,num_words_n,num_hash_n,num_mention_n,num_url_n,has_location,geocoded,longitude_n,latitude_n,target,keyword,location,text,cleaned_text,wordvec,keyword_wordvec
0,1,0.433121,0.419355,0.076923,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this earthquake Ma...,"[-0.26623327, 0.05843069, -0.1404636, -0.05265...","[-0.26623327, 0.05843069, -0.1404636, -0.05265..."
1,4,0.242038,0.225806,0.000000,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask. Canada,"[-0.025449565, 0.031005142, -0.15566371, -0.23...","[-0.025449565, 0.031005142, -0.15566371, -0.23..."
2,5,0.847134,0.709677,0.000000,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,All residents asked to 'shelter in place' are ...,All residents asked to 'shelter in place' are ...,"[0.0059339865, 0.016337818, -0.105279535, -0.0...","[0.0059339865, 0.016337818, -0.105279535, -0.0..."
3,6,0.356688,0.225806,0.076923,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,"13,000 people receive #wildfires evacuation or...",people receive wildfires evacuation orders in ...,"[-0.18147185, 0.20731743, 0.014147284, -0.2182...","[-0.18147185, 0.20731743, 0.014147284, -0.2182..."
4,7,0.541401,0.516129,0.153846,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,Just got sent this photo from Ruby Alaska as s...,"[-0.06394094, -0.01423019, 0.0063574947, 0.071...","[-0.06394094, -0.01423019, 0.0063574947, 0.071..."


## Putting all data in one SVM model
Puts metafeatures and features in one big SVM model.

In [33]:
train_normalized.columns

Index(['id', 'num_char_n', 'num_words_n', 'num_hash_n', 'num_mention_n',
       'num_url_n', 'has_location', 'geocoded', 'longitude_n', 'latitude_n',
       'target', 'keyword', 'location', 'text', 'cleaned_text', 'wordvec',
       'keyword_wordvec'],
      dtype='object')

In [40]:
import numpy
def get_X(df, col_wv, cols_ext):
    X = []
    for index, row in df.iterrows():
        x = row[col_wv]
        x = numpy.append(row[cols_ext].values, x)
        X.append(x)        
    return X

In [43]:
cols_ext = ['num_char_n', 'num_words_n', 'num_hash_n', 'num_mention_n',
       'num_url_n', 'has_location', 'geocoded', 'longitude_n', 'latitude_n']
X = get_X(train_normalized, 'wordvec',cols_ext)
y = train_normalized['target']

In [44]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svm = SVC(kernel="rbf")
params = {'C': [0.05, 0.1, 0.2, 0.5, 0.75, 1, 1.5, 2, 4]}
clf = GridSearchCV(svm, params, scoring="f1", verbose=1, n_jobs=-2, cv=5)

In [45]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf, X, y, cv=cv, return_train_score=True, scoring='f1')
scores

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  2.5min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  3.0min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  4.2min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  4.2min finished


{'fit_time': array([158.19189024, 189.40911245, 242.12806582, 265.95231915,
        270.01944232]),
 'score_time': array([2.1467371 , 2.26172423, 3.16707468, 3.3029592 , 3.25709343]),
 'test_score': array([0.76399027, 0.74728941, 0.7749577 , 0.76101695, 0.77564637]),
 'train_score': array([0.82279274, 0.79297365, 0.81854249, 0.78786602, 0.79731149])}

In [46]:
scores['test_score'].mean()

0.7645801390849052

slight improvement?

In [25]:
def prepare_submission(model, X, y, X_test, name):
    fit = model.fit(X,y)
    pred = model.predict(X_test)
    submission = pd.DataFrame({"id":test['id'], "target":pred})
    submission.to_csv(name+'.csv', index=False)
    return fit

In [48]:
X_test = get_X(test_normalized, 'wordvec',cols_ext)

In [49]:
fit = prepare_submission(clf, X, y, X_test, 'avg_wordvec_meta_svm')

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  3.8min finished


In [50]:
fit.best_params_

{'C': 0.5}

Not a public score improvement

## Stacking an average wordvecs model and and a metafeatures model

In [94]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import TransformerMixin, BaseEstimator
import numpy

In [95]:
class UnpackWordvecTransformer(BaseEstimator, TransformerMixin):
    def transform(self, df, **transform_params):
        X = []
        for index, row in df.iterrows():
            x = row['wordvec']
            #print(len(x))
            X.append(x)    
        return numpy.array(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [96]:
cols_ext = ['num_char', 'num_words', 'num_hash', 'num_mention', 'num_url',
            'has_location', 'geocoded', 'longitude_t', 'latitude_t']
t = ColumnTransformer(transformers=[('identity', 'passthrough', cols_ext)])

In [98]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
params = {'max_depth': [1,2,3,4], 'n_estimators':[5,10,20,40,80]}
lg = LGBMClassifier(n_jobs=-2, random_state=42, max_depth=1, n_estimators=20)

In [99]:
pipe_meta = Pipeline(steps=[('sel', t),
                           ('rf', lg)])

In [100]:
t.fit(train_tree)

ColumnTransformer(transformers=[('identity', 'passthrough',
                                 ['num_char', 'num_words', 'num_hash',
                                  'num_mention', 'num_url', 'has_location',
                                  'geocoded', 'longitude_t', 'latitude_t'])])

In [101]:
from sklearn.neural_network import MLPClassifier
twordvec = ColumnTransformer(transformers=[('wv', UnpackWordvecTransformer(), ['wordvec'])])

In [102]:
twordvec.fit(train_tree)

ColumnTransformer(transformers=[('wv', UnpackWordvecTransformer(),
                                 ['wordvec'])])

In [103]:
mlp = MLPClassifier(random_state=42, max_iter=2000, hidden_layer_sizes=(2,), learning_rate_init=5e-05)
pipe_nlp = Pipeline(steps=[('sel',twordvec),
                           ('mlp',mlp)    
])

In [104]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(max_iter = 1000)

In [41]:
from sklearn.ensemble import StackingClassifier
sc = StackingClassifier(estimators=[('meta',pipe_meta), ('nlp',pipe_nlp)], final_estimator=lg)

In [42]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_validate(sc, train_tree, train_tree['target'], cv=cv, return_train_score=True, scoring='f1', n_jobs=-2)
scores

{'fit_time': array([1196.22317004, 1014.42060447, 1249.54983878, 1115.1602962 ,
        1027.11536098]),
 'score_time': array([0.72195482, 0.4821713 , 1.53263855, 0.70491838, 0.69602585]),
 'test_score': array([0.76456504, 0.75701684, 0.76960388, 0.769107  , 0.76812749]),
 'train_score': array([0.86564763, 0.86231884, 0.81956088, 0.86046979, 0.79418211])}

In [43]:
scores['test_score'].mean()

0.7656840507805377

In [44]:
X = train_tree.drop(columns=['target','id'])
X_test = test_tree.drop(columns=['id'])

In [45]:
prepare_submission(sc, X, train_tree['target'], X_test, 'stack_meta_mlp')

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   40.4s finished


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   39.5s finished


Fitting 5 folds for each of 20 candidates, totalling 100 fits


...

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:  1.0min finished


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   52.1s finished


StackingClassifier(estimators=[('meta',
                                Pipeline(steps=[('sel',
                                                 ColumnTransformer(transformers=[('wv',
                                                                                  UnpackWordvecTransformer(),
                                                                                  ['wordvec'])])),
                                                ('rf',
                                                 GridSearchCV(cv=5,
                                                              estimator=LGBMClassifier(n_jobs=-2,
                                                                                       random_state=42),
                                                              n_jobs=-2,
                                                              param_grid={'max_depth': [1,
                                                                                        2,
                          

Did not improve public score..

## Stacking an average wordvecs model, a bow model and a metafeatures model

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
tbow = ColumnTransformer(transformers=[('bow', CountVectorizer(), 'cleaned_text')])

In [106]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter = 1000)

In [107]:
pipe_bow = Pipeline(steps=[('sel',tbow),
                           ('clf',clf)    
])

In [108]:
twordvec = ColumnTransformer(transformers=[('wv', UnpackWordvecTransformer(), ['wordvec'])])

In [109]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svm = SVC(kernel="rbf",C=1)
pipe_wordvec_svm = Pipeline(steps=[('sel',twordvec),
                           ('svm',svm)    
])

In [110]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(max_iter = 1000)

In [111]:
from sklearn.ensemble import StackingClassifier
sc = StackingClassifier(estimators=[('meta',pipe_meta), 
                                    ('bow',pipe_bow), 
                                    ('wordvec_mlp',pipe_nlp), 
                                    ('wordvec_svm',pipe_wordvec_svm)], 
                        final_estimator=lg)

In [112]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_validate(sc, train_tree, train_tree['target'], cv=cv, return_train_score=True, scoring='f1', n_jobs=-2)
scores

{'fit_time': array([466.78894591, 370.93477273, 480.38580751, 344.68219972,
        320.62175894]),
 'score_time': array([3.93788195, 2.66316319, 4.17244959, 2.60680461, 2.61485982]),
 'test_score': array([0.75974026, 0.76253081, 0.76744186, 0.76846473, 0.76324503]),
 'train_score': array([0.79293139, 0.79426434, 0.80231118, 0.80808081, 0.79097436])}

In [113]:
scores['test_score'].mean()

0.764284539416835

In [68]:
X = train_tree.drop(columns=['target','id'])
X_test = test_tree.drop(columns=['id'])

In [69]:
prepare_submission(sc, X, train_tree['target'], X_test, 'stack_meta_avgvec_bow')

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   41.1s finished


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   43.9s finished


Fitting 5 folds for each of 20 candidates, totalling 100 fits


...

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   50.7s finished


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:   52.2s finished


StackingClassifier(estimators=[('meta',
                                Pipeline(steps=[('sel',
                                                 ColumnTransformer(transformers=[('wv',
                                                                                  UnpackWordvecTransformer(),
                                                                                  ['wordvec'])])),
                                                ('rf',
                                                 GridSearchCV(cv=5,
                                                              estimator=LGBMClassifier(n_jobs=-2,
                                                                                       random_state=42),
                                                              n_jobs=-2,
                                                              param_grid={'max_depth': [1,
                                                                                        2,
                          